In [102]:
### Installing the required packages if not already installed
packages = ['numpy', 'pandas', 'warnings', 'sqlite3', 'scipy', 'matplotlib', 'numba']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        %pip install {package}

import numpy as np    # For numerical computing
import pandas as pd   # For data manipulation
import sqlite3        # For connecting to SQL database
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy import stats
from numba import njit

### Setting the random seed for reproducibility
np.random.seed(42)

### Ignoring the warnings
import warnings
warnings.filterwarnings('ignore')

### Pandas display options
pd.options.display.float_format = '{:.4f}'.format

### Getting some fuctions from the other notebook
%run 99_functions.ipynb

In [103]:
### Extracting the data from the database
conn = sqlite3.connect('Data/data.db')

factor_ret_m = pd.read_sql('SELECT * FROM factors_monthly', conn).set_index('Date').drop(columns='rf')
volatility = pd.read_sql('SELECT * FROM volatility', conn).set_index('Date')
volatility_timed_factors = pd.read_sql('SELECT * FROM volatility_timed_factors', conn).set_index('Date')
volatility_managed_ptf = pd.read_sql('SELECT * FROM volatility_managed_factor_ptf_outofsample', conn).set_index('Date')

conn.close()

### Making sure all the data is in the same format
factor_ret_m.index = pd.to_datetime(factor_ret_m.index)
volatility.index = pd.to_datetime(volatility.index)
volatility_managed_ptf.index = pd.to_datetime(volatility_managed_ptf.index)
volatility_timed_factors.index = pd.to_datetime(volatility_timed_factors.index)

### Matching the index of factor_ret_m to volatility_timed_factors
factor_ret_m = factor_ret_m.loc[volatility_timed_factors.index]

In [104]:
### Creating volatility timed factors using only the market volatility
volatility_timed_factors_mkt = pd.DataFrame(index=volatility_timed_factors.index, columns=volatility_timed_factors.columns)
market_vol = volatility['VW'].shift(1)

### Calculating the optimal c for each factor
c = pd.DataFrame(columns=volatility.columns)
c.loc['c', :] = np.nan

for factor in volatility.columns:
    def obj_func(c):
        vol_timed = (c / market_vol) * factor_ret_m[factor]
        return (vol_timed.std() - factor_ret_m[factor].std())**2

    res = minimize(obj_func, 0.1)
    c.loc['c', factor] = res.x[0]

### Calculating the volatility timed factors
for factor in volatility_timed_factors_mkt.columns:
    volatility_timed_factors_mkt[factor] = c.loc['c', factor] / market_vol * factor_ret_m[factor]

### Print
print(c)
print(round(volatility_timed_factors_mkt.std() - factor_ret_m.std()).sum())

     SMB    HML    UMD     EW     VW    BAB
c 0.1690 0.1710 0.1515 0.1599 0.1546 0.1472
0.0


In [ ]:
### Combining the factor returns
factor_ext = pd.concat([factor_ret_m, volatility_timed_factors], axis=1)
factor_ext_mkt = pd.concat([factor_ret_m, volatility_timed_factors_mkt], axis=1)

### Adding _vol to the last 6 columns
factor_ext.columns = [col + '_org' for col in factor_ext.columns[:-6]] + [col + '_vol' for col in factor_ext.columns[-6:]]
factor_ext_mkt.columns = [col + '_org' for col in factor_ext_mkt.columns[:-6]] + [col + '_vol' for col in factor_ext_mkt.columns[-6:]]

### Creating a dataframe to store the portfolios in
MV = pd.DataFrame(index=factor_ext.index, columns=['CMV_fac', 'CMV_mkt', 'UMV'])

### Creating a place to store the weights in to see what is going on
weights_org = pd.DataFrame(index=factor_ret_m.index, columns=factor_ret_m.columns)
weights_ext = pd.DataFrame(index=factor_ext.index, columns=factor_ext.columns)
weights_mkt = pd.DataFrame(index=factor_ext.index, columns=factor_ext.columns)

### Running the loop for mean-variance optimization
w = 120
for t in range(w, len(factor_ext)):
    # Selecting the data for the last w months
    data_org = factor_ret_m.iloc[t-w:t]
    data_ext = factor_ext.iloc[t-w:t]
    data_mkt = factor_ext_mkt.iloc[t-w:t]
    ret_org = factor_ret_m.iloc[t]
    ret_ext = factor_ext.iloc[t]
    ret_mkt = factor_ext_mkt.iloc[t]

    # Getting the weights
    w_org = ptf_weights(data_org, negative=False, type = "MVP", gamma = 5)
    w_ext = ptf_weights(data_ext, negative=False, type = "MVP", gamma = 5)
    w_mkt = ptf_weights(data_mkt, negative=False, type = "MVP", gamma = 5)

    # Storing the weights
    weights_org.loc[factor_ret_m.index[t], :] = w_org
    weights_ext.loc[factor_ext.index[t], :] = w_ext
    weights_mkt.loc[factor_ext.index[t], :] = w_mkt

    # Calculating the returns
    MV.loc[factor_ext.index[t], 'UMV'] = np.dot(ret_org, w_org)
    MV.loc[factor_ext.index[t], 'CMV_fac'] = np.dot(ret_ext, w_ext)
    MV.loc[factor_ext.index[t], 'CMV_mkt'] = np.dot(ret_mkt, w_mkt)

### Removing rows with only NaN
MV = MV.dropna(how='all')

### Looking at the sharpe ratios
sharpes_ratio(MV)

CMV       2.1288
CMV_mkt   2.2157
UMV       2.0242
dtype: object

In [108]:
### Getting the p-value of the sharpe ratio for CMV being bigger then UMV
n_boot = 100000
SR_diff_fac = []
SR_diff_mkt = []

### Running the loop
for i in range(n_boot):
    boot = stationary_bootstrap_multivariate(np.array(MV.astype(float)), 5, len(MV))
    SR_diff_fac.append(sharpes_ratio(boot[:,2]) - sharpes_ratio(boot[:,0]))
    SR_diff_mkt.append(sharpes_ratio(boot[:,2]) - sharpes_ratio(boot[:,1]))

### Calculating the p-value
p_value_fac = (np.sum(np.array(SR_diff_fac) > 0)) / (n_boot)
p_value_mkt = (np.sum(np.array(SR_diff_mkt) > 0)) / (n_boot)

### Printing
print('The p-value for the difference in Sharpe Ratios between CMV_fac and UMV is:', p_value_fac)
print('The p-value for the difference in Sharpe Ratios between CMV_mkt and UMV is:', p_value_mkt)

The p-value for the difference in Sharpe Ratios between CMV_fac and UMV is: 0.28386
The p-value for the difference in Sharpe Ratios between CMV_mkt and UMV is: 0.0138


In [109]:
### Storing the data
conn = sqlite3.connect('Data/data.db')

MV.to_sql('MV', conn, if_exists='replace')

conn.close()